In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(latex2exp)
}))


In [29]:
# input

sumstats_f <- 'INI50.lst'
var_anno_cal_f <- '/oak/stanford/groups/mrivas/ukbb24983/cal/variant_filter_table.tsv.gz'
metal_f <- '/oak/stanford/groups/mrivas/users/ytanigaw/sandbox/20200612_metal_power_analysis/INI50.%s.metal.tsv.gz'


In [12]:
sumstats_list <- fread(sumstats_f, head=F, sep='\t') %>% pull()
sumstats_list <- setNames(
    sumstats_list[1:7],
    c('WB','NBW','Afr','SA','EA','rel','others')
)


In [4]:
fread(var_anno_cal_f, colClasses=c('CHROM'='character')) %>%
filter(ld_indep) %>% pull(ID) -> ld_indep_lst


In [24]:
read_ld_indep_vars <- function(in_f, ld_indep_lst){
    in_f %>%
    fread(colClasses=c('#CHROM'='character')) %>%
    rename('CHROM'='#CHROM') %>%
    filter(ID %in% ld_indep_lst) -> df    
    if('ERRCODE' %in% colnames(df)){
        df %>% filter(is.na(ERRCODE) |  ERRCODE=='.') -> df
    }
    df %>%
    select(CHROM, POS, ID, BETA, SE, P)
}


In [7]:
qq_plot <- function(df){
    df %>%
    mutate(log10P = -log10(as.numeric(P))) %>% 
    arrange(-log10P) %>% drop_na(log10P) %>% pull(log10P) -> qq_p_obs
    
    -(qq_p_obs %>% length() %>% ppoints() %>% log10()) -> qq_p_exp
    
    p <- data.frame(
        Observed = qq_p_obs,
        Expected = qq_p_exp
    ) %>% ggplot(aes(x = Expected, y = Observed)) + geom_point() + 
    geom_abline(slope=1, intercept=0, color='red') + 
    theme_bw() + 
    labs(
        y = TeX('Observed $-\\log_{10}(P)$'),
        x = TeX('Expected $-\\log_{10}(P)$')
    )    
}


In [25]:
plots <- list()
for(pop in names(sumstats_list)){
    sumstats_list[[ pop ]] %>% 
    read_ld_indep_vars(ld_indep_lst) %>% 
    qq_plot() -> plots[[ pop ]] 
}


In [19]:
ggsave('3_qq_WB.png', plots[['WB']] + labs(title = 'GWAS Q-Q plot for WB, Standing height (INI50)'))


Saving 6.67 x 6.67 in image


In [18]:
names(sumstats_list)

[1] "WB"     "NBW"    "Afr"    "SA"     "EA"     "rel"    "others"

In [27]:
g <- gridExtra::arrangeGrob(
    plots[['NBW']] + labs(title = 'GWAS Q-Q plot for NBW, INI50'),
    plots[['Afr']] + labs(title = 'GWAS Q-Q plot for African, INI50'),
    plots[['SA']]  + labs(title = 'GWAS Q-Q plot for South Asian, INI50'),
    plots[['EA']]  + labs(title = 'GWAS Q-Q plot for East Asian, INI50'),
    plots[['others']]  + labs(title = 'GWAS Q-Q plot for others, INI50'),
    plots[['rel']]  + labs(title = 'GWAS Q-Q plot for semi-related, INI50'),
    ncol=3
)


In [28]:
ggsave('3_qq.png', g, width=12, height=8)


In [30]:
sprintf(metal_f, '7pops') %>%
read_ld_indep_vars(ld_indep_lst) %>% 
qq_plot() -> p_Metal


In [32]:
ggsave('3_qq_Metal_7pops.png', p_Metal + labs(title = 'Meta-analysis GWAS (7 pops) Q-Q plot, Standing height (INI50)'))


Saving 6.67 x 6.67 in image
